# LoadGaugesGH.ipynb

## Load gauge time series from netCDF file

Under development for the [Cascadia CoPes Hub](https://cascadiacopeshub.org/) project, supported by NSF.

:::{note}
DRAFT version with limited data -- only 4 ground motions

Adapted from [](../Newport/LoadGaugesNewport.ipynb)
:::

This notebook uses a datafile containing gauge time series the 36 ground motions developed for the 
Cascadia CoPes Hub, at gauge locations near Westport, WA, for a 2-hour simulation using [GeoClaw](https://www.geoclaw.org) with 1/3 arcsecond resolution on the finest grid level.

This data file can be found from the Design Safe Jupyter Hub in
  ~/MyProjects/PRJ-6005/GraysHarborBridges
  
The gauge locations are on an interactive map that can be created in this notebook.

Data used can be found on DesignSafe in 

    ~/MyProjects/PRJ-6005/GraysHarborBridges
    
or downloaded to your local machine via:

    scp username@stampede3.tacc.utexas.edu:/corral/projects/NHERI/projects/7f2e74be-d7ca-4e0e-b69a-22c24840b078/GraysHarborBridges/*.nc ./
    

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import xarray
import os,sys,glob
from pathlib import Path
from IPython.display import FileLink

In [ ]:
sys.path.insert(0,'../../src')
import CHTuser

In [ ]:
from CHTuser import CHTtools

## Read the netCDF file containing all time series

Make sure `ncfile` includes the correct path to the netCDF file with gauge results.

In [ ]:
pwd

In [ ]:
if 0:
    # on Design Safe
    datadir = '~/MyProjects/PRJ-6005/GraysHarborBridges'
else:
    # local data
    datadir = '.'

# full path:
datadir = Path(datadir).resolve()

print(f'netcdf files in {datadir}:')
glob.glob(f'{datadir}/*.nc')

Set `location` to one of the locations in the directories above.

In [ ]:

location = 'OceanShores'
ncfile = f'{datadir}/{location}_gauges_4events.nc'

print(f'ncfile = {ncfile}')

In [ ]:
gauge_x, gauge_y, gauge_t, gauge_vals = CHTtools.read_allgauges_nc(ncfile)

### Examine the data:

In [ ]:
gauge_vals

## Convert to an xarray dataset

`gauge_vals` is one large 4-dimensional array indexed by the quantity of interest `qoi` as well as by `event`, `gaugeno`, and `time`.
For most purposes it's easiest  to convert is to an `xarray.Dataset`, a collection of DataArray's split up by the qoi:

In [ ]:
ds = gauge_vals.to_dataset(dim='qoi')

In [ ]:
ds

In [ ]:
all_events = ds.coords['event'].data
all_gauges = ds.coords['gaugeno'].data

In [ ]:
print('\nEvents: \n',all_events)
print('\nGauges: \n',all_gauges)
print(f'\nAt {len(ds.coords['time'])} times up to {array(ds.coords['time']).max()/3600} hours\n')

In [ ]:
h = array(ds.h.sel(gaugeno=544, event='BL10D_instant'))
h[:3]

### Make an html file with interactive map showing all gauges:

Still need to clean up this function and move to `CHTtools`....

In [ ]:
def folium_plot_gauge(gaugenos, center=None, zoom_start=13):
    import folium
    import numpy as np

    if not isinstance(gaugenos, (list, tuple, np.ndarray)):
        # If it's not a list, wrap it in one
        gaugenos = [gaugenos]
       
    #tiles = 'OpenStreetMap'  # default tiles
    tiles = 'OpenTopoMap'   # show contours
    
    m = None

    for gaugeno in gaugenos:
 
        xg = gauge_x.sel(gaugeno=gaugeno)
        yg = gauge_y.sel(gaugeno=gaugeno)
        #print(f'Gauge %i is at (%.5f, %.5f)' % (gaugeno, xg, yg))

        if m is None:
            if center is None:
                location=(yg, xg)
            elif isinstance(center,int):
                xgc = gauge_x.sel(gaugeno=center)
                ygc = gauge_y.sel(gaugeno=center)
                location = (ygc, xgc)
            else:
                #assume center is (y,x) location
                location = center
            
            m = folium.Map(location=location, tiles=tiles, zoom_start=zoom_start)
        
        folium.Marker(
            location=[yg,xg],
            #popup = f"<b>Gauge {gaugeno}</b>\n ({xg:.6f},\n{yg:.6f})",
            popup = f"<b>Gauge {gaugeno}</b>\n {yg:.6f}N\n{-xg:.6f}W",
            tooltip="Click for info",
            icon=folium.Icon(color="red")  #, icon="cloud") # Customize the marker's appearance
        ).add_to(m) 
        
    return m

In [ ]:
all_gauges = ds.coords['gaugeno'].data
map_with_all_gauges = folium_plot_gauge(all_gauges, center=None, zoom_start=12)
fname = f'{location}_bridges_gauges_folium_map.html'
map_with_all_gauges.save(fname)
print('Created ',fname)

# to display the map here, uncomment the next line:
#map_with_all_gauges

FileLink(fname)

In [ ]:
if 0:
    # plot one gauge on interactive map
    mygaugeno = all_gauges[0]
    map = folium_plot_gauge(mygaugeno)  #  argument can be single gauge or list of gauges
    print(f'Location of Gauge {mygaugeno}:')
    map

## Adding new qoi's

Now we can refer to `ds.h`, for example, to get the `h` variable (rather than having to use `gauge_vals.sel(qoi='h')`.

We can also easily add new quantities of interest to the dataset that are computed from others, e.g. the momentum flux.


In [ ]:
ds['momflux'] = ds.h * (ds.u**2 + ds.v**2)

In [ ]:
momflux_max = float(ds.momflux.max())
print(f'ds.momflux has shape {ds.momflux.shape}')
print(f'   and the maximum value over all events / gauges /times is {momflux_max:.2f} m^3/s^2')

## Filtering out coarse-grid values

With the AMR strategy now being used, the finest level grids are not added near the gauges until the tsunami is arriving. Plots of the water depth before this time may show non-physical discontinuities at times where the AMR level changed, since on a coarse grid the topography is different and the water depth may be deeper or shallower as a result. (Plotting the surface level `eta = h+B`, where `B` is the topography value, would give smoother results at offshore gauges but might show discontinuities at onshore gauges.  See [this example](https://rjleveque.github.io/geoclaw_tsunami_tutorial/GTT/CopalisBeach/example1/gauges.html) for more discussion.)

The gauge quantity `ds.level` contains the AMR level from which the time series data was captured at each time, so we can use this to filter our coarse grid times and define a new variable `hfine` that is masked at these points and only has data from the finest level:

In [ ]:
hfine = ma.masked_where(ds.level != ds.level.max(), ds.h)

In [ ]:
ds['hfine'] = (('time','gaugeno','event'),hfine)

### Some gauges never had fine grids!

For the `WestportBridges` location, some gauges never got wet and the AMR algorithms never refined around them, so filtering out coareser grids leaves nothing to plot. 

Since these gauges are all on dry land, do not do the filter for the plots below...

## Select one gauge for one event

In [ ]:
mygaugeno = all_gauges[0]
myevent = all_events[0]
xg = gauge_x.sel(gaugeno=mygaugeno)
yg = gauge_y.sel(gaugeno=mygaugeno)
print(f'Gauge {mygaugeno} is at x = {xg:.6f}, y = {yg:.6f}')

### Plot all the data and also the filtered data from the finest AMR level only:

In [ ]:
hfine = ds.hfine.sel(gaugeno=mygaugeno, event=myevent)
h = ds.h.sel(gaugeno=mygaugeno, event=myevent)
figure(figsize=(10,6))
tminutes = gauge_t / 60.
    
plot(tminutes, h, 'r',label='all data')
plot(tminutes, hfine, 'b',label='finest AMR level')

grid(True)
legend(loc='upper right', framealpha=1)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,xg,yg,myevent));

## Plot all events at one gauge

Plotting `h` rather than `hfine`

In [ ]:
all_events = ds.coords['event'].data  # all events
h_all = ds.h.sel(gaugeno=mygaugeno, event=all_events)

figure(figsize=(10,6))
for ev in all_events:
    plot(tminutes, h_all.sel(event=ev), label=ev)
legend()  # not very useful with 36 events
grid(True)
xlim(20,8*60)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %s' % mygaugeno);

### What's the maximum value at this gauge over all events, and which event is largest?

From the plot above it's hard to tell which event corresponds to the upper-most curve, so let's look at the maximum depth at this gauge for each event:

In [ ]:
print('The maximum depth over all time for all events is %.2f meters' % h_all.max())

We can plot the maximum depth for each event separately:

In [ ]:
hmax = h_all.max(dim='time')  # 1D array of 36 values for each event

figure(figsize=(6,3))
ievents = range(len(hmax.coords['event']))
plot(hmax.data, ievents)
yticks(ievents, hmax.coords['event'].data);
grid(True)
xlabel('meters')
title('Maximum water depth over 90 minutes at Gauge %i' % mygaugeno);

A more programatic way to determine which event has this maximum value is to use `argmax`:

In [ ]:
imax = int(hmax.argmax(dim='event'))
emax = all_events[imax]
hmax = float(ds.h.sel(event=emax, gaugeno=mygaugeno).max())
print(f'Event {imax} named {emax} has the largest maximum depth at Gauge {mygaugeno} with value {hmax:.2f} m')

### Plot this largest event to confirm:

In [ ]:
figure(figsize=(10,6))
tminutes = gauge_t / 60.
plot(tminutes, h_all.sel(event=emax), 'b')
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,xg,yg,emax));

## Plot all gauges for all events

For a small number of events and gauges, this is reasonable...

In [ ]:
all_events = ds.coords['event'].data  # all events
all_gauges = ds.coords['gaugeno'].data

for gaugeno in all_gauges:
    h_all = ds.h.sel(gaugeno=gaugeno, event=all_events)
    
    figure(figsize=(10,4))
    for ev in all_events:
        plot(tminutes, h_all.sel(event=ev), label=ev)
    legend()  # not very useful with 36 events
    grid(True)
    xlim(0,8*60)
    xlabel('minutes')
    ylabel('meters')
    title('Water depth at Gauge %s' % gaugeno);

## hazard curves

In [ ]:
hmh0.coords['event'].data

In [ ]:
mygaugeno = 154
myevent = 'BL10D_instant'

In [ ]:
def probs(hazard_qoi, exceed_vals, gaugeno, weights, events=None):
    if events is None:
        events = hazard_qoi.coords['event'].data  # all avaliable events
    pvals = zeros(exceed_vals.shape)
    for j in range(len(exceed_vals)):
        for eventk in events:
            if eventk not in hazard_qoi.coords['event']:
                raise ValueError(f'*** event {eventk} not available in hazard_qoi')
            if hazard_qoi.sel(gaugeno=gaugeno, event=eventk) > exceed_vals[j]:
                pk = weights.sel(event=eventk)
                pvals[j] = pvals[j] + pk - pvals[j]*pk
    return pvals

In [ ]:
h0 = ds.h.sel(time=0)
hmh0 = ds.h.max(dim='time') - h0
hmh0

In [ ]:
hmh0test = float(hmh0.sel(gaugeno=mygaugeno, event=myevent))
print(f'At gauge {mygaugeno}, event {myevent}, hmh0 = {hmh0test:.3f}')

In [ ]:
figure(figsize=(6,3))
ievents = range(len(ds.coords['event']))
hmax = hmh0.sel(gaugeno=mygaugeno)
plot(hmax, ievents)
yticks(ievents, hmax.coords['event'].data);
grid(True)
xlabel('meters')
title('Maximum h-h0 over 90 minutes at Gauge %i' % mygaugeno);

In [ ]:
weights = xarray.DataArray([0.0375, 0.0625, 0.025, 0.0375], dims=('event'),
                           coords={'event':ds.coords['event']})
print(f'The weight for event {myevent} is {float(weights.sel(event=myevent))}')

In [ ]:
exceed_vals = arange(0,8,0.1)
mygaugeno = 544
events = None # for all events
events = ['BL10D_instant', 'BL13D_instant']
pvals = probs(hmh0, exceed_vals, mygaugeno, weights, events)
plot(exceed_vals,pvals,'g', label='Deep events')
events = ['BL10M_instant', 'BL10S_instant']
pvals = probs(hmh0, exceed_vals, mygaugeno, weights, events)
plot(exceed_vals,pvals,'b', label='Other events')
pvals = probs(hmh0, exceed_vals, mygaugeno, weights, events=None)
plot(exceed_vals,pvals,'r--', label='All 4 events')
grid(True)
legend(loc='upper right', framealpha=1)
title(f'Hazard curve of max(h-h0) for Gauge {mygaugeno}');
xlabel('exceedance value (m)')
ylabel('probability of exceedance')
savefig('sample_hazard_curve.png')

In [ ]:
for e,p in zip(exceed_vals,pvals):
    print(f'{e:.3f},  {p:.6f}')